<a href="https://colab.research.google.com/github/caroldaza/DM_CyT/blob/Raul/Calculo_distancias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# We'll need to update our path to import from Drive.
import sys
sys.path.append('/content/drive/MyDrive/procesados')

## Importa paquetes



In [5]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt 
import matplotlib.gridspec as gridspec
from scipy.signal import butter, lfilter
from sklearn import decomposition

Bajo paquete para trabajar con EEG- Me permite ver la topografía

In [6]:
!pip install mne
import mne

     |████████████████████████████████| 6.9 MB 19.4 MB/s 


## (1) Levanto datos de EEG

El registro de EEG de 30 canales viene en formato txt. La frecuencia de muestreo es de 128 Hz.

Los datos ya vienen limpios y filtrados entre 2 y 20 Hz

Topología


***HAY QUE CALCULAR AQUI ALGUNA DE LAS MEDIDAS DE DISTANCIA VISTAS EN CLASE PARA COMPARAR LAS CABECITAS***
y hay que revisar la función que dibuja para hacerlas más bonitas

Qué tan distintos son los individuos entre sí, vistos en cada periodo?
Hallaremos el promedio de la medición en cada canal durante el primer segundo (
1 segundo) en el primer bloque de reposo (antes de la tarea de aprendizaje)

# Mediciones en el primer bloque (en reposo previo a la tarea de aprendizaje)
Calcularemos el promedio de las mediciones de cada sujeto durante el primer segundo (es decir, durante las primeras 128 mediciones (por ser el sample rate = 128))



In [110]:
promedios_bl_1_1s = []
# df = pd.DataFrame()
subject = [8,9,10,12,13] 
srate=128 
inicio = 1 
fin    = 2  
for ind,sub in enumerate(subject):  
    filename1 = '/content/drive/MyDrive/procesados/suj_' + str(sub) + '_bl_1.txt'        
    datos1 = pd.read_csv(filename1, sep=',', header=None)    
    EEGdata1_1seg = datos1.to_numpy()   
    EEGdata1_1seg = EEGdata1_1seg[:,inicio*srate:fin*srate]        
    medias_fila = EEGdata1_1seg.mean(axis=1)    
    medias_fila.shape
    # medias_fila.transpose
    # print(medias_fila)    
    promedios_bl_1_1s.append(medias_fila)

    print(sub)

8
9
10
12
13


In [111]:
kept_channels = ['Fp1', 'Fp2', 'F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'O1', 'O2', 'F7', 'F8', 
                 'T7', 'T8', 'P7', 'P8', 'Fz', 'Cz', 'Pz', 'FC1', 'FC2',
                 'FC5', 'FC6','CP1','CP2','CP5','CP6','AFz','Fpz','POz']

df = pd.DataFrame(promedios_bl_1_1s, columns= kept_channels, dtype = float) 
df

,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,F7,F8,T7,T8,P7,P8,Fz,Cz,Pz,FC1,FC2,FC5,FC6,CP1,CP2,CP5,CP6,AFz,Fpz,POz
0,-0.797705,-0.432769,0.312478,0.113405,0.190725,-0.287079,-0.237071,0.109291,-0.536993,0.001848,-0.034502,-0.058565,-0.049159,-0.036872,0.237431,-0.067748,0.013553,0.437186,-0.187127,0.016881,0.514832,0.374645,-0.277238,0.656085,0.451375,0.070416,0.372480,-0.104407,-0.742257,-0.023064
1,-0.071581,-0.090876,-0.166033,0.158866,-0.148120,-0.693563,-0.973258,-0.417496,0.364423,-0.104969,-0.067546,0.459020,-0.028478,-0.361922,0.571312,0.493086,-0.029986,0.121003,0.529532,-0.294255,0.061621,0.361293,-0.077623,0.324555,0.321459,-0.243295,-0.417581,-0.226157,0.387720,0.258854
2,-0.120162,-0.094653,0.217349,0.328558,0.444406,-0.358509,-0.262834,0.133283,-0.749788,-0.472076,-0.153180,0.270873,0.478316,-0.341106,0.023237,-0.116288,-0.314957,0.292998,0.704498,-0.068330,-0.286321,-0.317592,-0.603629,0.510797,0.861830,0.064205,0.033236,-0.320650,0.604743,-0.388247
3,0.153329,-0.220583,-0.104933,-0.037169,0.757687,0.398052,0.064125,0.617179,0.059903,-0.251638,-0.133616,-0.447152,-0.040771,0.492697,0.394079,-0.400880,-0.159631,-0.346783,-0.544431,1.601226,-0.404815,-0.265025,-0.099902,-0.088917,0.172401,-0.366826,0.118671,-0.181651,-0.102295,-0.632349
4,-0.054113,0.034422,-0.172250,-0.056821,0.108050,-0.065845,0.333202,0.253173,-0.014472,-0.211112,-0.369092,0.104665,0.297137,-0.053285,0.084684,0.107527,-0.256957,-0.159325,0.112077,-0.155037,0.350376,-0.170270,-0.153912,0.204988,0.000789,-0.052602,0.036569,-0.338847,0.322875,-0.066600


## Cálculo de la distancia euclidea

In [102]:
import scipy

ary = scipy.spatial.distance.cdist(df.iloc[:,1:], df.iloc[:,1:], metric='euclidean')
ary.round(3)

array([[0.   , 2.5  , 2.347, 2.936, 2.032],
       [2.5  , 0.   , 2.438, 3.721, 2.194],
       [2.347, 2.438, 0.   , 3.213, 1.991],
       [2.936, 3.721, 3.213, 0.   , 2.633],
       [2.032, 2.194, 1.991, 2.633, 0.   ]])

## Cálculo de la distancia de Manhattan 

In [115]:
# Import necessary package
import scipy.spatial.distance as d

def CityBlock_distance(A, B):

	result = d.cdist(A, B, 'cityblock')
	return result

manhattan_dist = CityBlock_distance(df, df)
print(manhattan_dist.round(3))


[[ 0.    11.552 10.029 13.835  9.682]
 [11.552  0.    11.089 15.373  9.422]
 [10.029 11.089  0.    14.028  8.78 ]
 [13.835 15.373 14.028  0.    10.511]
 [ 9.682  9.422  8.78  10.511  0.   ]]


# Mediciones en el segundo bloque (en reposo posterior a la tarea de aprendizaje)
Calcularemos el promedio de las mediciones de cada sujeto durante el primer segundo

In [116]:
promedios_bl_2_1s = []
# df = pd.DataFrame()
subject = [8,9,10,12,13] 
srate=128 
inicio = 1 
fin    = 2  
for ind,sub in enumerate(subject):  
    filename1 = '/content/drive/MyDrive/procesados/suj_' + str(sub) + '_bl_2.txt'
    datos1 = pd.read_csv(filename1, sep=',', header=None)    
    EEGdata1_1seg = datos1.to_numpy()   
    EEGdata1_1seg = EEGdata1_1seg[:,inicio*srate:fin*srate]        
    medias_fila = EEGdata1_1seg.mean(axis=1)    
    medias_fila.shape
    # medias_fila.transpose
    # print(medias_fila)    
    promedios_bl_2_1s.append(medias_fila)

8
9
10
12
13


In [117]:
df = pd.DataFrame(promedios_bl_2_1s, columns= kept_channels, dtype = float) 
df

,Fp1,Fp2,F3,F4,C3,C4,P3,P4,O1,O2,F7,F8,T7,T8,P7,P8,Fz,Cz,Pz,FC1,FC2,FC5,FC6,CP1,CP2,CP5,CP6,AFz,Fpz,POz
0,-0.261031,0.758761,0.156790,-0.137884,-0.276327,0.126293,0.214401,-0.140864,0.085400,-0.091247,-0.146566,-0.254963,-0.248971,-1.133349,0.088116,-0.084110,0.250853,0.072357,-0.279628,-0.188215,0.309438,0.482672,0.174575,0.734761,0.086673,-0.235157,-0.120334,-0.077146,0.377369,-0.242631
1,0.389912,0.289716,0.000849,-0.268204,-0.101493,0.667330,0.206725,-0.109399,0.183974,0.432120,0.041981,-0.453087,-0.128648,-0.293847,-0.053315,-0.390979,0.321405,-0.071681,-0.339163,0.075867,0.240389,0.371730,-0.116010,-0.283232,-0.239231,-0.191432,-0.265693,0.308076,-0.348734,0.124072
2,-0.168786,0.295848,0.327207,0.214130,0.351656,0.199542,0.143895,0.385427,0.352714,0.443766,0.175417,0.324032,0.250799,0.358815,0.392940,0.250986,0.182410,0.243532,0.175952,0.194104,0.732321,-0.143860,-8.069397,0.054430,0.204837,0.489944,0.253264,0.398309,0.342747,0.642832
3,0.273957,0.095297,0.149924,0.300229,0.029143,0.791282,0.595909,0.417171,0.268643,-0.090082,-0.226152,-0.651317,-1.011168,0.071549,-0.004537,-0.794736,-0.044737,-0.172572,0.045978,0.785627,-0.219315,-0.132316,0.117397,-0.062027,0.220102,-0.376580,0.383933,-0.498116,0.000512,-0.263015
4,-0.135491,0.003960,-0.183279,0.144577,-0.052563,-0.156379,-0.281447,0.085448,-0.117664,-0.160659,-0.005644,0.218164,0.136670,-0.129009,-0.017598,0.211677,-0.070329,0.024729,0.105615,0.334960,0.038591,0.021239,-0.146639,0.046961,0.048871,0.050503,0.301260,-0.251935,0.083821,-0.148397


## Cálculo de la distancia euclidea

In [118]:
import scipy

ary = scipy.spatial.distance.cdist(df.iloc[:,1:], df.iloc[:,1:], metric='euclidean')
ary.round(3)

array([[0.   , 1.988, 8.695, 2.734, 2.125],
       [1.988, 0.   , 8.295, 2.295, 2.083],
       [8.695, 8.295, 0.   , 8.724, 8.168],
       [2.734, 2.295, 8.724, 0.   , 2.444],
       [2.125, 2.083, 8.168, 2.444, 0.   ]])

## Cálculo de la distancia de Manhattan 

In [119]:
# Import necessary package
import scipy.spatial.distance as d

def CityBlock_distance(A, B):

	result = d.cdist(A, B, 'cityblock')
	return result

manhattan_dist = CityBlock_distance(df, df)
print(manhattan_dist.round(3))

[[ 0.     8.598 20.374 12.419  9.736]
 [ 8.598  0.    19.394 10.789 10.113]
 [20.374 19.394  0.    21.08  16.62 ]
 [12.419 10.789 21.08   0.     9.786]
 [ 9.736 10.113 16.62   9.786  0.   ]]
